In [ ]:
import os
import time
import gc
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn

In [ ]:
# Device configuration
if torch.cuda.is_available():
    print("Device Use: GPU")
    device = torch.device("cuda")
else :
    print("Device Use: CPU")
    device = torch.device("cpu")

In [ ]:
# Hyper-parameters 
num_epochs = 5
batch_size = 10
learning_rate = 0.001
momentum = 0.005

In [ ]:
data_path = "../input/vr-project-data/dataset"
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

In [ ]:
img_size=100
data=[]
target=[]

for category in categories:
    
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        
        img_path = os.path.join(folder_path,img_name)
        img = cv.imread(img_path)
        
        #Coverting the image into gray scale
        gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)            

        #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
        resized=cv.resize(gray,(img_size,img_size),interpolation = cv.INTER_CUBIC)            

        #appending the image and the label(categorized) into the list (dataset)
        data.append(resized)
        target.append(label_dict[category])

In [ ]:
data=np.array(data).astype(np.float32)
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],1,img_size,img_size))
target=np.array(target).astype(np.int)

images = torch.from_numpy(data)
label = torch.from_numpy(target)

print(images.shape)
print(label.shape)

# createing data set
total = torch.utils.data.TensorDataset(images,label)
train = torch.utils.data.DataLoader(total, batch_size=batch_size, shuffle=True)

dataiter = iter(train)
images,label = dataiter.next()
print(images.shape)
print(label.shape)

del data
del target
gc.collect()

In [ ]:
# conv1 = nn.Conv2d(1,100,3)
# ac = nn.ReLU()
# pool = nn.MaxPool2d(2)

# conv2 = nn.Conv2d(100,200,3)
# ac = nn.ReLU()
# pool = nn.MaxPool2d(2)

# # x = x.view(x.size(0), -1)
# drop = nn.Dropout(0.2)

# print("Original: ", images.shape)

# x=conv1(images)
# print("conv1: ",x.shape)

# x=pool(x)
# print("Pool: ",x.shape)

# x=conv2(x)
# print("conv2: ",x.shape)

# x=pool(x)
# print("Pool: ",x.shape)

# x=drop(x)
# print("Drop: ",x.shape)

# # x=pool(x)
# # print("Pool: ",x.shape)

# fc1 = nn.Linear(32 * 3 * 3, 50)
# fc2 = nn.Linear(50, 2)

In [ ]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()        
        
        self.ac = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        
        self.conv1 = nn.Conv2d(1,100,3)
        self.conv2 = nn.Conv2d(100,200,3)
        self.drop = nn.Dropout(0.2)
        
        self.fc1 = nn.Linear(200*23*23, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        
        #cnn layer 1
        x = self.conv1(x)
        x = self.ac(x)
        x = self.pool(x)
        
        #cnn layer 2
        x = self.conv2(x)
        x = self.ac(x)
        x = self.pool(x)
        
        #flatten
        x = x.view(-1,200*23*23)
        
        #drop
        x = self.drop(x)
        
        #dense
        x = self.fc1(x)
        
        #dense
        x = self.fc2(x)
        return x

model = ConvNet().to(device)
criterion =  nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate ,momentum = momentum)

In [ ]:
start_time = time.time()

for epoch in range(num_epochs):  
    tot_loss=0.0
    for i, (images, labels) in enumerate(train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tot_loss+=loss.item()
    
    print ("Epoch: ", epoch+1, " total Loss: ", tot_loss , "Average Loss : ", tot_loss/len(train) )

end_time = time.time()            
print('Finished Training')
print('Time taken:',end_time - start_time)

print(model.eval())
PATH = './model.pth'
torch.save(model, PATH)

In [ ]:
mm = torch.load(PATH)
print(mm.eval())
print("Model Saved!!")